In [1]:
!pip install transformers
!pip install torch
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 78.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 93.3 M

Imports:

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Set filepaths for text file and csv file:

In [2]:
# text_filepath = "../home/schen9/dataset.txt"
text_filepath = "dataset.txt"
csv_filepath = "dataset.csv"

Read in the text file and split it by double newline:

In [3]:
with open(text_filepath, "r", encoding="utf-8", errors="ignore") as text_file:
    rows = text_file.read().strip().split("\n\n")

print(f"Finished reading from {text_filepath}") 

Finished reading from ../home/schen9/dataset.txt


Write the text to the csv file:

In [4]:
import csv

with open(csv_filepath, "w", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Text"]) # Header
    for i, row in enumerate(tqdm(rows)):
        writer.writerow([row.strip()])

print(f"Finished writing to {csv_filepath}") 

  0%|          | 0/449919 [00:00<?, ?it/s]

Finished writing to dataset.csv


Load the csv onto a hugging face dataset object:

In [5]:
from datasets import Dataset, load_dataset

dataset = load_dataset("csv",data_files=csv_filepath) 
dataset = dataset.filter(lambda x: x["Text"] is not None) # filter out NoneTypes
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/449919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 446203
    })
})

Load the tokenizer:

In [6]:
from transformers import AutoTokenizer

transformer_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

Tokenize all the text in batches:

In [7]:
def tokenize(batch):
    return tokenizer(batch["Text"], truncation=True)

batch_size = 50
tokens = dataset.map(tokenize, batched=True, batch_size=batch_size)
tokens

Map:   0%|          | 0/446203 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text', 'input_ids', 'attention_mask'],
        num_rows: 446203
    })
})

In [8]:
tokens.save_to_disk("my_tokens")

Saving the dataset (0/2 shards):   0%|          | 0/446203 [00:00<?, ? examples/s]

In [9]:
df = tokens["train"].to_pandas()
df

,Text,input_ids,attention_mask
0,The White Monkey is a 1925 American silent dra...,"[0, 133, 735, 34546, 16, 10, 36248, 470, 8454,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Preservation\nAn incomplete print of The White...,"[0, 28917, 26481, 50118, 4688, 20044, 5780, 9,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,External links,"[0, 47380, 5678, 2]","[1, 1, 1, 1]"
3,Films based on works by John Galsworthy\nFilms...,"[0, 36361, 4339, 716, 15, 1364, 30, 610, 272, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,the montane grasslands and shrublands biome\n ...,"[0, 627, 27121, 1728, 6964, 8391, 8, 15383, 17...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...
446198,References,"[0, 49379, 2]","[1, 1, 1]"
446199,External links,"[0, 47380, 5678, 2]","[1, 1, 1, 1]"
446200,"1929 births\n2015 deaths\nPeople from Naseby, ...","[0, 1646, 2890, 26906, 50118, 14420, 3257, 501...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
446201,Scottish female golfers\nGolfers from Edinburg...,"[0, 22041, 1173, 2182, 3524, 268, 50118, 534, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [10]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(transformer_name, output_hidden_states = True).to(device)
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [11]:
def get_embeddings(t_vector, a_mask, model):
    token_tensor = torch.tensor([t_vector]).to(model.device)
    seg_tensor = torch.tensor([a_mask]).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=token_tensor, attention_mask=seg_tensor)
        hidden_state = outputs[1]
    token_embeddings = torch.stack(hidden_state, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs.append(sum_vec)
    return token_vecs

# Iterate over rows in DataFrame and call the function
token_counts = {i: 0 for i in range(tokenizer.vocab_size)}
avg_embeddings = {}

for t_vector, a_mask in tqdm(zip(df["input_ids"], df["attention_mask"]),total=len(df)):
    embeddings = get_embeddings(t_vector, a_mask, model)
    for t_index, token in enumerate(t_vector):
        token_counts[token] += 1
        embedding = embeddings[t_index] # Get the corresponding embedding for the token
        if token not in avg_embeddings:
            avg_embeddings[token] = embedding
        else:
            avg_embeddings[token] = torch.add(embedding, avg_embeddings[token])
    # break  # Break after first iteration for testing

# Average the embeddings
for token in avg_embeddings.keys():
    avg_embeddings[token] = torch.div(avg_embeddings[token], token_counts[token])

  0%|          | 0/446203 [00:00<?, ?it/s]

/tmp/ipykernel_1832/765774988.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  token_tensor = torch.tensor([t_vector]).to(model.device)


In [12]:
import pickle 

with open('avg_embeddings.pkl', 'wb') as f:
    pickle.dump(avg_embeddings, f)